# "Plan your trip with Kayak"
## _Data Collection and Management Project_

### 1. Récupérer les données météo

Grâce à `nominatim.streetmap.org`, nous allons pouvoir obtenir les coordonnées GPS d'une sélection de villes.

In [ ]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv("./.env"))

In [3]:
def get_cities_coordinates(cities: list[str]) -> list[dict]:
    url = "https://nominatim.openstreetmap.org"
    endpoint = "search"
    data = []
    for city in cities:
        city = city.lower()
        payload = {"city": city, "format": "json"}
        response = requests.get(f"{url}/{endpoint}", params=payload)

        if response.status_code != 200:
            print(f"Failed with '{city}' -> {response.status_code}")
            continue

        resp = response.json()[0]
        data.append({"city": city,
                        "lat": resp.get("lat", None),
                        "lon": resp.get("lon", None)})

    return data

Toujours via `requests`, nous allons cette fois-ci récupérer des données sur la météo à venir dans une sélection de villes.
Pour cela, utilisons l'API `openweathermap.org`.

In [ ]:

def get_weather_data(cities_coordinates: list[dict]) -> list[dict]:
    url = "https://api.openweathermap.org/data/2.5"
    endpoint = "forecast"
    data = []
    city_id = 1
    for row in cities_coordinates.copy():
        payload = {"lat": float(row["lat"]),
                    "lon": float(row["lon"]),
                    "units": "metric",
                    "cnt": 7,  # 7 days to come
                    "appid": os.getenv("WEATHER_KEY")}

        response = requests.get(f"{url}/{endpoint}",
                                params=payload)

        if response.status_code != 200:
            print(f"Failed -> {response.status_code}")
            continue

        day_id = 1
        for dt in response.json()["list"]:
            data.append({
                "city_id": city_id,
                "city": row["city"],
                "lat": float(row["lat"]),
                "lon": float(row["lon"]),
                "day_id": day_id,
                "temp": dt["main"].get("temp", None),
                "temp_min": dt["main"].get("temp_min", None),
                "temp_max": dt["main"].get("temp_max", None),
                "humidity": dt["main"].get("humidity", None),
                "clouds": dt["clouds"].get("all", None),
                "rain_prob": dt.get("pop", None),
            })
            day_id += 1
        city_id += 1

    return data

Maintenant que les fonctions sont écrites, il n'y a plus qu'à enregistrer le résultat dans un fichier CSV !
_(Pour des raisons de gains de place, les conventions PEP8 ne sont pas respectées dans la cellule ci-dessous)._

In [ ]:
cities = ["city_01", "city_02"]

# Save to CSV file
df = pd.DataFrame(get_weather_data(get_cities_coordinates(cities)))
df.to_csv("weather_data.csv", index=False, encoding="utf-8")

### 2. Récupérer les données sur les hôtels

Le client veut se baser sur le site booking.com pour collecter ses données. Comme il n'y a pas d'API officielle, il va falloir faire du web scraping pour obtenir les informations dont nous avons besoin.